# Importações básicas

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Iniciar a sessão Spark

In [2]:
spark = SparkSession.builder\
                    .config("spark.master","local[*]")\
                    .config("spark.jars.packages", "org.postgresql:postgresql:42.3.4")\
                    .appName("Tranformações - Aula x")\
                    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/home/aluno/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/aluno/.ivy2/cache
The jars for the packages stored in: /home/aluno/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-006e6f76-c5d9-4dff-b94c-5ecb375d3c2a;1.0
	confs: [default]
	found org.postgresql#postgresql;42.3.4 in central
	found org.checkerframework#checker-qual;3.5.0 in central
:: resolution report :: resolve 273ms :: artifacts dl 11ms
	:: modules in use:
	org.checkerframework#checker-qual;3.5.0 from central in [default]
	org.postgresql#postgresql;42.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------------------

# Módulo(s) de Obtenção de Dados

## Carga de Arquivos - Compras Públicas

- utilização de esquema **INFERIDO**

In [3]:
compraInferSchemaDF = spark.read\
                            .format("csv")\
                            .option("sep", ";")\
                            .option("header",True)\
                            .option("charset","iso-8859-1")\
                            .option("inferSchema",True)\
                            .load("/home/aluno/_spark/dados/originais/compras_publicas_federal/compras")
                            #.option("dateFormat", "dd/MM/yyyy")\ ## Esse não funciona no InferSchema
                           

In [4]:
compraInferSchemaDF.show()

+------------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+------------+--------------------+---------+--------------------+------------------------+-------------------+--------------------+-----------------+---------------+--------------------+--------------------+------------------+----------------+-------------------+--------------------+----------------------------------+---------------------------+
|Número do Contrato|              Objeto|    Fundamento Legal|   Modalidade Compra|   Situação Contrato|Código Órgão Superior| Nome Órgão Superior|Código Órgão|          Nome Órgão|Código UG|             Nome UG|Data Assinatura Contrato|Data Publicação DOU|Data Início Vigência|Data Fim Vigência|CNPJ Contratado|     Nome Contratado|Valor Inicial Compra|Valor Final Compra|Número Licitação|Código UG Licitação|   Nome UG Licitação|Código Modalidade Compra Licitação|Modalidade Compra Licitação|
+---------

In [5]:
compraInferSchemaDF.printSchema()

root
 |-- Número do Contrato: integer (nullable = true)
 |-- Objeto: string (nullable = true)
 |-- Fundamento Legal: string (nullable = true)
 |-- Modalidade Compra: string (nullable = true)
 |-- Situação Contrato: string (nullable = true)
 |-- Código Órgão Superior: integer (nullable = true)
 |-- Nome Órgão Superior: string (nullable = true)
 |-- Código Órgão: integer (nullable = true)
 |-- Nome Órgão: string (nullable = true)
 |-- Código UG: integer (nullable = true)
 |-- Nome UG: string (nullable = true)
 |-- Data Assinatura Contrato: string (nullable = true)
 |-- Data Publicação DOU: string (nullable = true)
 |-- Data Início Vigência: string (nullable = true)
 |-- Data Fim Vigência: string (nullable = true)
 |-- CNPJ Contratado: string (nullable = true)
 |-- Nome Contratado: string (nullable = true)
 |-- Valor Inicial Compra: string (nullable = true)
 |-- Valor Final Compra: string (nullable = true)
 |-- Número Licitação: integer (nullable = true)
 |-- Código UG Licitação: integer 

In [6]:
# Boa Prática - Nome das colunas sem acento e sem espaços.
compraInferSchemaDF.select(
                            F.col("Número do Contrato").alias("nr_cont"),
                            F.col("Objeto").alias("obj"),
                            F.col("Fundamento Legal").alias("fund_legal"),
                            F.col("Modalidade Compra").alias("modld_comp"),
                            F.col("Situação Contrato").alias("sit_contrt"),
                            F.col("Código Órgão Superior").alias("cod_org_sup")
                          ).show()

+-------+--------------------+--------------------+--------------------+--------------------+-----------+
|nr_cont|                 obj|          fund_legal|          modld_comp|          sit_contrt|cod_org_sup|
+-------+--------------------+--------------------+--------------------+--------------------+-----------+
|  92019|Objeto: Aquisição...|Fundamento Legal:...|Dispensa de Licit...|           Publicado|      26000|
|  12020|Objeto: Contrataç...|Fundamento Legal:...|Pregão - Registro...|           Publicado|      52000|
|  22020|Objeto: Serviços ...|Fundamento Legal:...|              Pregão|           Publicado|      26000|
|5012020|Objeto: Contrataç...|Fundamento Legal:...|Inexigibilidade d...|           Publicado|      52000|
| 512020|Objeto: Prestação...|Fundamento Legal:...|              Pregão|           Publicado|      39000|
|  52019|Objeto: Contrataç...|Fundamento Legal:...|    Tomada de Preços|Rescindido - Publ...|      53000|
| 202019|Objeto: Aquisição...|Fundamento Legal

## Carga de Arquivos 02 - Compras Públicas

- Utilização de esquema **MANUAL**

### Criação do Esquema schemaCompras

In [7]:
# importe dos tipos de dados para criação do esquema manual
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, LongType, DateType, DoubleType, DecimalType

schemaCompra = StructType().add(StructField("nr_cont", IntegerType()))\
                           .add(StructField("obj", StringType()))\
                           .add(StructField("fund_leg", StringType()))\
                           .add(StructField("mod_comp", StringType()))\
                           .add(StructField("sit_cont", StringType()))\
                           .add(StructField("cod_org_sup", IntegerType()))\
                           .add(StructField("nm_org_sup", StringType()))\
                           .add(StructField("cod_org", IntegerType()))\
                           .add(StructField("nm_org", StringType()))\
                           .add(StructField("cod_ug", IntegerType()))\
                           .add(StructField("nm_ug", StringType()))\
                           .add(StructField("dt_ass_cont", DateType()))\
                           .add(StructField("dt_pub_dou", DateType()))\
                           .add(StructField("dt_ini_vig", DateType()))\
                           .add(StructField("dt_fim_vig", DateType()))\
                           .add(StructField("cnpj_contrtd", LongType()))\
                           .add(StructField("nm_contrtd", StringType()))\
                           .add(StructField("vl_ini_comp", StringType()))\
                           .add(StructField("vl_fim_comp", StringType()))\
                           .add(StructField("nr_lic", IntegerType()))\
                           .add(StructField("cod_ug_lic", IntegerType()))\
                           .add(StructField("nm_ug_lic", StringType()))\
                           .add(StructField("cod_mod_comp_lic", IntegerType()))\
                           .add(StructField("desc_mod_comp_lic", StringType()))
# embora a natureza dos campos seja decimal, o spark não consegue converter o padrão separado por virgula
# no momento da carga do dados, por isto os campos >>vl_ini_comp e vl_fim_comp<< foram configurados como string(texto)

### Carga do DataFrame

In [8]:
compraEsquemaManualDF = spark.read\
                             .format("csv")\
                             .option("sep",";")\
                             .option("header","true")\
                             .option("charset","iso-8859-1")\
                             .schema(schemaCompra)\
                             .option("dateFormat", "dd/MM/yyyy")\
                             .load("/home/aluno/_spark/dados/originais/compras_publicas_federal/compras/")

In [9]:
compraEsquemaManualDF.show()
compraEsquemaManualDF.printSchema()

+-------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+-------+--------------------+------+--------------------+-----------+----------+----------+----------+--------------+--------------------+------------+------------+---------+----------+--------------------+----------------+--------------------+
|nr_cont|                 obj|            fund_leg|            mod_comp|            sit_cont|cod_org_sup|          nm_org_sup|cod_org|              nm_org|cod_ug|               nm_ug|dt_ass_cont|dt_pub_dou|dt_ini_vig|dt_fim_vig|  cnpj_contrtd|          nm_contrtd| vl_ini_comp| vl_fim_comp|   nr_lic|cod_ug_lic|           nm_ug_lic|cod_mod_comp_lic|   desc_mod_comp_lic|
+-------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+-------+--------------------+------+--------------------+-----------+----------+----------+----------+--------------+---------------

<h4 style="color:yellow">Exercício 01: criação de esquemas e carga de csv</h4>

a) Criar dois esquemas manuais que atendam aos requisitos a seguir:
    schemaItemCompra
    schemaTermoAditivo

In [10]:
# importe dos tipos de dados para criação do esquema manual
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, LongType, DateType, DoubleType, DecimalType

schemaItemCompra = StructType().add(StructField("cod_org", IntegerType()))\
                               .add(StructField("nm_org", StringType()))\
                               .add(StructField("cod_ug", IntegerType()))\
                               .add(StructField("nm_ug", StringType()))\
                               .add(StructField("nr_cont", IntegerType()))\
                               .add(StructField("cod_item_comp", DecimalType()))\
                               .add(StructField("desc_item_comp", StringType()))\
                               .add(StructField("desc_compl_item_comp", StringType()))\
                               .add(StructField("qtd_item", IntegerType()))\
                               .add(StructField("vl_item", StringType()))

In [11]:
schemaTermoAditivo = StructType().add(StructField("nr_cont", IntegerType()))\
                               .add(StructField("cod_org_sup", IntegerType()))\
                               .add(StructField("nm_org_sup", StringType()))\
                               .add(StructField("cod_org", IntegerType()))\
                               .add(StructField("nm_org", StringType()))\
                               .add(StructField("cod_ug", IntegerType()))\
                               .add(StructField("nm_ug", StringType()))\
                               .add(StructField("nr_term_adit", IntegerType()))\
                               .add(StructField("dt_pub", DateType()))\
                               .add(StructField("obj", StringType()))

b) Utilizandos os esquemas criados, carrega os dados dos arquivos localizados nos caminhos

In [14]:
itemCompraManualDF = spark.read\
                          .format("csv")\
                          .option("sep",";")\
                          .option("header","true")\
                          .option("charset","iso-8859-1")\
                          .schema(schemaItemCompra)\
                          .option("dateFormat", "dd/MM/yyyy")\
                          .load("/home/aluno/_spark/dados/originais/compras_publicas_federal/compras/itens_compra/")

In [15]:
itemCompraManualDF.show()
itemCompraManualDF.printSchema()

+-------+--------------------+------+--------------------+-------+-------------+--------------------+--------------------+--------+----------+
|cod_org|              nm_org|cod_ug|               nm_ug|nr_cont|cod_item_comp|      desc_item_comp|desc_compl_item_comp|qtd_item|   vl_item|
+-------+--------------------+------+--------------------+-------+-------------+--------------------+--------------------+--------+----------+
|  52121| Comando do Exército|160100|COMANDO DA 3A BDA...|  12018|         null|ANALISE CLINICA, ...|PRESTAÇÃO DE SERV...|       1| 120000,00|
|  30108|Departamento de P...|200366|DELEGACIA DE POLI...|  12019|            0|Informação proteg...|Informação proteg...|     600|      6,00|
|  30108|Departamento de P...|200366|DELEGACIA DE POLI...|  12019|            0|Informação proteg...|Informação proteg...|      30| 153510,00|
|  30108|Departamento de P...|200366|DELEGACIA DE POLI...|  12019|            0|Informação proteg...|Informação proteg...|     600| 596118,00|

In [16]:
termosAditivosManualDF = spark.read\
                          .format("csv")\
                          .option("sep",";")\
                          .option("header","true")\
                          .option("charset","iso-8859-1")\
                          .schema(schemaTermoAditivo)\
                          .option("dateFormat", "dd/MM/yyyy")\
                          .load("/home/aluno/_spark/dados/originais/compras_publicas_federal/temos_aditivo/")

In [17]:
termosAditivosManualDF.show()
termosAditivosManualDF.printSchema()

+--------+-----------+--------------------+-------+--------------------+------+--------------------+------------+----------+--------------------+
| nr_cont|cod_org_sup|          nm_org_sup|cod_org|              nm_org|cod_ug|               nm_ug|nr_term_adit|    dt_pub|                 obj|
+--------+-----------+--------------------+-------+--------------------+------+--------------------+------------+----------+--------------------+
|  742019|      20000|Presidência da Re...|  20101|Presidência da Re...|110001|SECRETARIA ESPECI...|       12020|2020-04-01|Objeto: Alterar o...|
|  752019|      20000|Presidência da Re...|  20101|Presidência da Re...|110001|SECRETARIA ESPECI...|       12020|2020-05-11|Objeto: Acréscimo...|
|  752019|      20000|Presidência da Re...|  20101|Presidência da Re...|110001|SECRETARIA ESPECI...|       22020|2020-12-21|Objeto: Prorrogaç...|
|   72019|      63000|Advocacia-Geral d...|  63000|Advocacia-Geral d...|110156|ESCOLA DA ADVOCAC...|       12020|2020-12-30|

Aula 17/09

In [19]:
compraEsquemaManualDF.show()

+-------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+-------+--------------------+------+--------------------+-----------+----------+----------+----------+--------------+--------------------+------------+------------+---------+----------+--------------------+----------------+--------------------+
|nr_cont|                 obj|            fund_leg|            mod_comp|            sit_cont|cod_org_sup|          nm_org_sup|cod_org|              nm_org|cod_ug|               nm_ug|dt_ass_cont|dt_pub_dou|dt_ini_vig|dt_fim_vig|  cnpj_contrtd|          nm_contrtd| vl_ini_comp| vl_fim_comp|   nr_lic|cod_ug_lic|           nm_ug_lic|cod_mod_comp_lic|   desc_mod_comp_lic|
+-------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+-------+--------------------+------+--------------------+-----------+----------+----------+----------+--------------+---------------

In [40]:
compraDF = compraEsquemaManualDF.withColumn("vl_ini_comp", F.regexp_replace("vl_ini_comp", ",", ".").cast(DoubleType()))\
                                .withColumn("vl_fim_comp", F.regexp_replace("vl_fim_comp", ",", ".").cast(DecimalType(16,2)))\
                                .select("nr_cont",
                                        "cod_ug", 
                                        "mod_comp", 
                                        "dt_ini_vig", 
                                        "vl_fim_comp"
                                       )
    
compraDF.show()
compraDF.printSchema()                         

+-------+------+--------------------+----------+-----------+
|nr_cont|cod_ug|            mod_comp|dt_ini_vig|vl_fim_comp|
+-------+------+--------------------+----------+-----------+
|  92019|158643|Dispensa de Licit...|2019-12-27|  230559.60|
|  12020|160224|Pregão - Registro...|2020-01-07|    6000.00|
|  22020|158526|              Pregão|2020-01-08|   17000.00|
|5012020|160173|Inexigibilidade d...|2020-01-01|  400000.00|
| 512020|393015|              Pregão|2020-02-03|  416143.91|
|  52019|193001|    Tomada de Preços|2020-01-23|  487018.68|
| 202019|240105|Dispensa de Licit...|2019-11-29|  119580.00|
| 362019|183038|              Pregão|2019-12-30| 6043946.05|
|  72019|160120|Pregão - Registro...|2019-12-11|   29463.00|
|  12020|155008|Pregão - Registro...|2020-01-14|  560810.00|
|  12020|154502|Dispensa de Licit...|2020-01-13|  928425.00|
|  12020|152005|              Pregão|2020-04-01|  144839.84|
| 792019|154050|Dispensa de Licit...|2019-10-31|    7631.36|
| 652019|155008|Inexigib

### Gravação do conteúdo de dataframe em arquivo CSV

In [42]:
compraDF.write\
        .mode("overwrite")\
        .option("header", True)\
        .option("sep", "\t")\
        .csv("home/aluno/_spark/dados/tratados/compra_publica.csv")

### Gravação do conteúdo de dataframe em JDBC

In [45]:
compraDF.write\
        .format("jdbc")\
        .mode("append")\
        .option("url", "jdbc:postgresql://127.0.0.1:5432/aula_fd")\
        .option("driver", "org.postgresql.Driver")\
        .option("dbtable", "compra_publica_teste")\
        .option("user", "aluno")\
        .option("password", "123")\
        .save()

### Gravação do conteúdo de dataframe no Metastore do Hive

#### Criação da base de dados no Metastore

In [51]:
spark.sql(
    """
        CREATE DATABASE aula_fd;
    """
)

AnalysisException: Database 'aula_fd' already exists

In [52]:
spark.sql(
    """
        show databases;
    """
).show()

+---------+
|namespace|
+---------+
|  aula_fd|
|  default|
+---------+



### Gravação do dataframe em uma tabela no Metastore

In [55]:
compraDF.write\
        .mode("append")\
        .saveAsTable("aula_fd.compra_publica")

#Grava o arquivo no padrão do Spark - formato parquet.

In [56]:
spark.sql(
    """
        SELECT * FROM aula_fd.compra_publica;
    """
).show()

+-------+------+--------------------+----------+-----------+
|nr_cont|cod_ug|            mod_comp|dt_ini_vig|vl_fim_comp|
+-------+------+--------------------+----------+-----------+
|  92019|158643|Dispensa de Licit...|2019-12-27|  230559.60|
|  12020|160224|Pregão - Registro...|2020-01-07|    6000.00|
|  22020|158526|              Pregão|2020-01-08|   17000.00|
|5012020|160173|Inexigibilidade d...|2020-01-01|  400000.00|
| 512020|393015|              Pregão|2020-02-03|  416143.91|
|  52019|193001|    Tomada de Preços|2020-01-23|  487018.68|
| 202019|240105|Dispensa de Licit...|2019-11-29|  119580.00|
| 362019|183038|              Pregão|2019-12-30| 6043946.05|
|  72019|160120|Pregão - Registro...|2019-12-11|   29463.00|
|  12020|155008|Pregão - Registro...|2020-01-14|  560810.00|
|  12020|154502|Dispensa de Licit...|2020-01-13|  928425.00|
|  12020|152005|              Pregão|2020-04-01|  144839.84|
| 792019|154050|Dispensa de Licit...|2019-10-31|    7631.36|
| 652019|155008|Inexigib

<h4 style="color:yellow">Exercício 02: gravação de dataframe outros formatos</h4>

**03 - gravar os dataframes carregados no exercício 02 em tabelas JDBC e em arquivos nos formatos**
- json **(local de gravação: /home/aluno/_spark/dados/exercicios/aula_fd/<nome_arquivo>.json)**
- orc **(local de gravação: /home/aluno/_spark/dados/exercicios/aula_fd/<nome_arquivo>.orc)**
- parquet **(local de gravação: /home/aluno/_spark/dados/exercicios/aula_fd/<nome_arquivo>.parquet)**
</br>

#### JDBC

In [61]:
itemCompraManualDF.write\
                    .format("jdbc")\
                    .mode("append")\
                    .option("url", "jdbc:postgresql://127.0.0.1:5432/aula_fd")\
                    .option("driver", "org.postgresql.Driver")\
                    .option("dbtable", "item_compra")\
                    .option("user", "aluno")\
                    .option("password", "123")\
                    .save()

In [62]:
termosAditivosManualDF.write\
                    .format("jdbc")\
                    .mode("append")\
                    .option("url", "jdbc:postgresql://127.0.0.1:5432/aula_fd")\
                    .option("driver", "org.postgresql.Driver")\
                    .option("dbtable", "termo_aditivo")\
                    .option("user", "aluno")\
                    .option("password", "123")\
                    .save()

#### JSON

In [63]:
itemCompraManualDF.write\
                        .mode("append")\
                        .option("header", True)\
                        .option("sep", "\t")\
                        .json("/home/aluno/_spark/dados/exercicios/aula_fd/.json")

In [65]:
termosAditivosManualDF.write\
                        .mode("append")\
                        .option("header", True)\
                        .option("sep", "\t")\
                        .json("/home/aluno/_spark/dados/exercicios/aula_fd/.json")

#### ORC

In [67]:
itemCompraManualDF.write\
                        .mode("append")\
                        .option("header", True)\
                        .option("sep", "\t")\
                        .orc("/home/aluno/_spark/dados/exercicios/aula_fd/.orc")

In [68]:
termosAditivosManualDF.write\
                        .mode("append")\
                        .option("header", True)\
                        .option("sep", "\t")\
                        .orc("/home/aluno/_spark/dados/exercicios/aula_fd/.orc")

#### PARQUET

In [69]:
itemCompraManualDF.write\
                        .mode("append")\
                        .option("header", True)\
                        .option("sep", "\t")\
                        .parquet("/home/aluno/_spark/dados/exercicios/aula_fd/.parquet")

In [70]:
termosAditivosManualDF.write\
                        .mode("append")\
                        .option("header", True)\
                        .option("sep", "\t")\
                        .parquet("/home/aluno/_spark/dados/exercicios/aula_fd/.parquet")

**04 - carregar arquivos gravados no exercício 03 em dataframes e observar as diferença em carregar os 
arquivos e carregar arquivo no formato CSV**

#### ItemCompra

In [87]:
itemCompraManualParquetDF = spark.read\
                          .parquet("/home/aluno/_spark/dados/exercicios/aula_fd/.parquet/")
itemCompraManualParquetDF.show()

+-------+--------------------+------+--------------------+-------+-------------+--------------------+--------------------+--------+----------+
|cod_org|              nm_org|cod_ug|               nm_ug|nr_cont|cod_item_comp|      desc_item_comp|desc_compl_item_comp|qtd_item|   vl_item|
+-------+--------------------+------+--------------------+-------+-------------+--------------------+--------------------+--------+----------+
|  52121| Comando do Exército|160100|COMANDO DA 3A BDA...|  12018|         null|ANALISE CLINICA, ...|PRESTAÇÃO DE SERV...|       1| 120000,00|
|  30108|Departamento de P...|200366|DELEGACIA DE POLI...|  12019|            0|Informação proteg...|Informação proteg...|     600|      6,00|
|  30108|Departamento de P...|200366|DELEGACIA DE POLI...|  12019|            0|Informação proteg...|Informação proteg...|      30| 153510,00|
|  30108|Departamento de P...|200366|DELEGACIA DE POLI...|  12019|            0|Informação proteg...|Informação proteg...|     600| 596118,00|

In [83]:
itemCompraManualOrcDF = spark.read\
                          .orc("/home/aluno/_spark/dados/exercicios/aula_fd/.orc/")

In [84]:
itemCompraManualJsonDF = spark.read\
                          .json("/home/aluno/_spark/dados/exercicios/aula_fd/.json/")

In [86]:
itemCompraManualJDBCDF = spark.read\
                        .format("jdbc")\
                        .mode("append")\
                        .option("url", "jdbc:postgresql://127.0.0.1:5432/aula_fd")\
                        .option("driver", "org.postgresql.Driver")\
                        .option("dbtable", "item_compra")\
                        .option("user", "aluno")\
                        .option("password", "123")\
                        .save()

AttributeError: 'DataFrameReader' object has no attribute 'mode'

#### Termo Aditivo

In [88]:
termosAditivosManualParquetDF = spark.read\
                          .parquet("/home/aluno/_spark/dados/exercicios/aula_fd/.parquet/")
termosAditivosManualParquetDF.show()

+-------+--------------------+------+--------------------+-------+-------------+--------------------+--------------------+--------+----------+
|cod_org|              nm_org|cod_ug|               nm_ug|nr_cont|cod_item_comp|      desc_item_comp|desc_compl_item_comp|qtd_item|   vl_item|
+-------+--------------------+------+--------------------+-------+-------------+--------------------+--------------------+--------+----------+
|  52121| Comando do Exército|160100|COMANDO DA 3A BDA...|  12018|         null|ANALISE CLINICA, ...|PRESTAÇÃO DE SERV...|       1| 120000,00|
|  30108|Departamento de P...|200366|DELEGACIA DE POLI...|  12019|            0|Informação proteg...|Informação proteg...|     600|      6,00|
|  30108|Departamento de P...|200366|DELEGACIA DE POLI...|  12019|            0|Informação proteg...|Informação proteg...|      30| 153510,00|
|  30108|Departamento de P...|200366|DELEGACIA DE POLI...|  12019|            0|Informação proteg...|Informação proteg...|     600| 596118,00|